## Dependencies

All the dependencies for this notebook are included in the `requirements.txt` file included in this folder.


In [1]:
import os
from pathlib import Path
import json

import datetime
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
from rasterio.plot import show

from IPython.display import clear_output

In [6]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

import tile_utils as tilu

In [3]:
channel_to_number = {"B01":0, "B02":1, "B03":2, "B04":3, "B05":4, "B06":5, "B07":6, "B08":7,
                     "B09":8, "B11":9, "B12":10, "B8A":11, "CLM":None, "BS_IDX": 12, "MOIST_IDX": 13, "NDVI_IDX": 14}
number_to_channel = {number: channel for channel, number in channel_to_number.items()}

In [4]:
def create_ndviindex(image, channel_to_number):
    image_B08 = image[channel_to_number['B08'],:,:]
    image_B04 = image[channel_to_number['B04'],:,:]
    non_zero = (image_B08 + image_B04 != 0)
    ndvi_index = np.zeros((image.shape[1], image.shape[2]))
    ndvi_index[non_zero] = np.nan_to_num((image_B08[non_zero] - image_B04[non_zero]) / (image_B08[non_zero] + image_B04[non_zero]))
    
    upper_limit = 1
    lower_limit = -0.2
    
    ndvi_index = np.where(ndvi_index > lower_limit, ndvi_index , lower_limit)
    ndvi_index = np.where(ndvi_index < upper_limit, ndvi_index , upper_limit)
    
    return ndvi_index

def create_moistureindex(image, channel_to_number):
    image_B8A = image[channel_to_number['B8A'],:,:]
    image_B11 = image[channel_to_number['B11'],:,:]
    non_zero = (image_B8A + image_B11 != 0)
    moisture_index = np.zeros((image.shape[1], image.shape[2]))
    moisture_index[non_zero] = np.nan_to_num((image_B8A[non_zero] - image_B11[non_zero]) / (image_B8A[non_zero] + image_B11[non_zero]))
    
    upper_limit = 2
    lower_limit = -0.8
    
    moisture_index = np.where(moisture_index > lower_limit, moisture_index , lower_limit)
    moisture_index = np.where(moisture_index < upper_limit, moisture_index , upper_limit)
    
    return moisture_index

def create_baresoilindex(image, channel_to_number):
    
    # Other option found: NBSI = ((B11 + B04)-(B08 + B02))/((B11 + B04)+(B08 + B02))
    
    image_B11 = image[channel_to_number['B11'],:,:]
    image_B08 = image[channel_to_number['B08'],:,:]
    image_B04 = image[channel_to_number['B04'],:,:]
    image_B02 = image[channel_to_number['B02'],:,:]
    
    non_zero = (image_B11 + image_B02 + image_B04 + image_B08 != 0)
    baresoil_index = np.zeros((image.shape[1], image.shape[2]))
    baresoil_index[non_zero] = np.nan_to_num(((image_B11[non_zero] + image_B04[non_zero]) - (image_B02[non_zero] + image_B08[non_zero])) / (image_B11[non_zero] + image_B02[non_zero] + image_B04[non_zero] + image_B08[non_zero]))
    
    return baresoil_index

def read_tile_geojson(path):
    tiles = gpd.read_file(path)
    for column in tiles.select_dtypes(include='object').columns:
        try:
            tiles[column] = tiles[column].apply(eval)
        except:
            pass
    return tiles

In [11]:
import os

def tile_dataset(orig_path, extra_data, tiles_info, train=True, radius=16):
    if train==True:
        path_app = ''
    else:
        path_app = '_test'
    
    tile_id = int(str(orig_path).split("/")[-1].split("'")[0])
    
    clear_output(wait=True)
    print(tile_id)
    
    list_tile = os.listdir(f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN{path_app}/{tile_id}/s2/')
    field_ids = rasterio.open(f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN{path_app}/{tile_id}/field_ids.tif').read(1)
    
    if train == True:
        labels = rasterio.open(f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN{path_app}/{tile_id}/labels.tif').read(1)
    
    days_tile = []
    
    for f in list_tile:
        days_tile.append(int(str(f).split("/")[-1].split("_")[-2]))
    days_tile = sorted(days_tile)
    
    field_ids_tile = list(np.unique(field_ids))
    field_ids_tile.remove(0)

    
    imgs = np.zeros((len(field_ids_tile),len(days_tile),15,radius*2,radius*2), dtype = np.float32)
    field_masks = np.zeros((len(field_ids_tile), 1, 1, radius*2, radius*2), dtype = np.float32)
    extra = np.zeros((len(field_ids_tile), 13))
    
    if train == True:
        field_labels = np.zeros((len(field_ids_tile),10))
    
    # load images for all days of this tile into dictionary
    img = dict()
    for num_day, day in enumerate(days_tile):
        img_path = f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN{path_app}/{tile_id}/s2/{tile_id}_s2_{day}_stacked.tif'         
        img[num_day] = rasterio.open(img_path).read() 
        
    num_neigh = len(eval(tiles_info.query('tile_id == 500').tiles_closest.iloc[0]))

    ifx = 0                   
    for field_id in field_ids_tile:
        
        field_mask = (field_ids==field_id)
        if train == True:
            field_labels[ifx,round(np.mean(labels[field_mask]))] = 1
            
        idxx = np.where(field_mask)
        momentx = np.median(idxx[0]).astype(np.int)
        momenty = np.median(idxx[1]).astype(np.int)

        field_patch = field_mask[max(0, momentx-radius): momentx+radius, max(0, momenty-radius): momenty+radius]

        
        if field_patch.sum() == 0:
            momentx = np.min(idxx[0]).astype(np.int)
            momenty = np.min(idxx[1]).astype(np.int)  
            field_patch = field_mask[max(0, momentx-radius): momentx+radius, max(0, momenty-radius): momenty+radius]
            
        if field_patch.sum() == 0:
            print('PATCH ZERO')
            print(ifx, momentx-radius, momentx+radius, momenty-radius, momenty+radius)
        

        for num_day, day in enumerate(days_tile):
            patch = img[num_day][0:12,max(0, momentx-radius): momentx+radius, max(0, momenty-radius): momenty+radius]

            imgs[ifx, num_day, 0:12, :patch.shape[-2], :patch.shape[-1]] = patch
            imgs[ifx, num_day, channel_to_number['MOIST_IDX'], :patch.shape[-2], :patch.shape[-1]] = create_moistureindex(patch, channel_to_number)
            imgs[ifx, num_day, channel_to_number['BS_IDX'], :patch.shape[-2], :patch.shape[-1]] = create_baresoilindex(patch, channel_to_number)
            imgs[ifx, num_day, channel_to_number['NDVI_IDX'], :patch.shape[-2], :patch.shape[-1]] = create_ndviindex(patch, channel_to_number)
        

        #pad crop's field mask in tiles borders with zeros
        field_masks[ifx, :, :, :patch.shape[-2], :patch.shape[-1]] = field_patch            
        extra[ifx, :12]  = np.array(extra_data.query('field_id == @field_id').drop('field_id', axis=1).values.tolist()[0])         
        extra[ifx, 12] = num_neigh
            
        ifx += 1
        
    if train == True:        
        return tf.data.Dataset.from_tensor_slices((imgs, field_masks, extra, field_labels))
    else:
        return tf.data.Dataset.from_tensor_slices((imgs, field_masks, extra)), field_ids_tile

In [12]:
tiles_dataset = tf.data.Dataset.list_files(f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN/*', shuffle=False)

fields_train_mean_var = pd.read_csv('data/mean_var_8days_train.csv')[['field_id', 'elevation', 'field_area_km2', 'sun_rate','neighboor_label_1', 'neighboor_label_2','neighboor_label_3',
                                'neighboor_label_4','neighboor_label_5', 'neighboor_label_6','neighboor_label_7', 'neighboor_label_8', 'neighboor_label_9']]

tiles_train = tilu.read_tile_geojson('data/tiles_train.geojson')
tiles_train_crs32634 = tiles_train.to_crs(32634)

# apply with a threshold of 4000m
tiles_train['tiles_closest'] = tiles_train_crs32634.apply(tilu.tiles_closest, axis='columns', args=(tiles_train_crs32634, 4000,))

In [13]:
for i, f in enumerate(tiles_dataset.as_numpy_iterator()):
    if i == 0:
        train_ds = tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var, tiles_info=tiles_train)
    elif i<2000:
        train_ds = train_ds.concatenate(tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var, tiles_info=tiles_train))
    elif i == 2000:
        val_ds = tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var, tiles_info=tiles_train)
    elif i<3000:
        val_ds = val_ds.concatenate(tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var, tiles_info=tiles_train))

999


In [296]:
for i, f in enumerate(tiles_dataset.as_numpy_iterator()):
    if i == 2100:
        val_ds = tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var)
    elif 2100 < i< 3000:
        val_ds = val_ds.concatenate(tile_dataset(f, train=True, radius=16, extra_data=fields_train_mean_var))

503
504
505
506
507
508
509
51
510
511
512
513
514
515
516
517
518
519
52
520
521
522
523
524
525
526
527
528
529
53
530
531
532
533
534
535
536
537
538
539
54
540
541
542
543
544
545
546
547
548
549
55
550
551
552
553
554
555
556
557
558
559
56
560
561
562
563
564
565
566
567
568
569
57
570
571
572
573
574
575
576
577
578
579
58
580
581
582
583
584
585
586
587
588
589
59
590
591
592
593
594
595
596
597
598
599
6
60
600
601
602
603
604
605
606
607
608
609
61
610
611
612
613
614
615
616
617
618
619
62
620
621
622
623
624
625
626
627
628
629
63
630
631
632
633
634
635
636
637
638
639
64
640
641
642
643
644
645
646
647
648
649
65
650
651
652
653
654
655
656
657
658
659
66
660
661
662
663
664
665
666
667
668
669
67
670
671
672
673
674
675
676
677
678
679
68
680
681
682
683
684
685
686
687
688
689
69
690
691
692
693
694
695
696
697
698
699
7
70
700
701
702
703
704
705
706
707
708
709
71
710
711
712
713
714
715
716
717
718
719
72
720
721
722
723
724
725
726
727
728
729
73
730
731
732
733
734

KeyboardInterrupt: 

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

batch_size = 1024

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.utils import plot_model

class DropMaskModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x = data[0]
        mask = data[1]
        extra = data[2]
        y = data[3]

        with tf.GradientTape() as tape:
            y_pred = self((x, mask, extra), training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    
    def test_step(self, data):
        # Unpack the data
        x = data[0]
        mask = data[1]
        extra = data[2]
        y = data[3]
        
        # Compute predictions
        y_pred = self((x, mask, extra), training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}
    
    def predict_step(self, data):
        """The logic for one inference step.
        This method can be overridden to support custom inference logic.
        This method is called by `Model.make_predict_function`.
        This method should contain the mathematical logic for one step of inference.
        This typically includes the forward pass.
        Configuration details for *how* this logic is run (e.g. `tf.function` and
        `tf.distribute.Strategy` settings), should be left to
        `Model.make_predict_function`, which can also be overridden.
        Args:
          data: A nested structure of `Tensor`s.
        Returns:
          The result of one inference step, typically the output of calling the
          `Model` on data.
        """
        x = data[0]
        mask = data[1]
        extra = data[2]

        return self((x, mask, extra), training=False)

In [18]:
checkpoint_path = "training_lstm2222_numneigh/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [54]:
cnn.load_weights(checkpoint_path)

In [20]:
cnn.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

In [25]:
train_complete_ds = train_ds.concatenate(val_ds)

In [ ]:
training = cnn.fit(train_complete_ds, validation_data=val_ds, epochs = 5, verbose=1, callbacks=[cp_callback])

Epoch 1/5
86/86 [==============================] - 625s 7s/step - loss: 0.9889 - accuracy: 0.6616 - val_loss: 1.1056 - val_accuracy: 0.6098

Epoch 00001: saving model to training_lstm2222_numneigh/cp.ckpt
Epoch 2/5
86/86 [==============================] - 625s 7s/step - loss: 0.9710 - accuracy: 0.6697 - val_loss: 1.1121 - val_accuracy: 0.5961

Epoch 00002: saving model to training_lstm2222_numneigh/cp.ckpt
Epoch 3/5
86/86 [==============================] - 625s 7s/step - loss: 0.9568 - accuracy: 0.6729 - val_loss: 1.0300 - val_accuracy: 0.6279

Epoch 00003: saving model to training_lstm2222_numneigh/cp.ckpt
Epoch 4/5
86/86 [==============================] - ETA: 0s - loss: 0.9389 - accuracy: 0.6783

In [19]:
def create_cnn_lstm(days, width, height, depth):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (days, depth, height, width)
    chanDim = [1, 2]
    
    # define the model input    
    img = tf.keras.Input(shape=inputShape, name='Input_Layer')
    mask = tf.keras.Input(shape=(1,1,32,32))
    extra = tf.keras.Input(shape=(13))
    
    
    x = tf.keras.layers.experimental.preprocessing.Normalization(axis=2)(img)
    
    conv_2d_layer = tf.keras.layers.Conv2D(32, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)

    conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)

    x = x * mask    
    x = tf.math.reduce_sum(x, axis = [3,4])
    mask_sum = tf.math.reduce_sum(mask, axis = [3,4])
    x = (x / mask_sum)
    
    x = tf.keras.layers.LSTM(64)(x)
    
    y = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(extra)
    y = BatchNormalization()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)

    x = Dense(128, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(10, kernel_initializer = 'HeUniform')(x)
    
    
    # construct the CNN
    model = DropMaskModel(inputs=[img, mask, extra], outputs=x)
    # return the CNN

    return model

cnn = create_cnn_lstm(8, 32, 32, 15)
#cnn.compile(optimizer = 'adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))
print(cnn.summary())

Model: "drop_mask_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 8, 15, 32, 3 0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 8, 15, 32, 32 31          Input_Layer[0][0]                
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 8, 32, 32, 32 4352        normalization[0][0]              
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 32, 32, 32 0           time_distributed[0][0]           
____________________________________________________________________________________

In [46]:
def create_cnn_temp_woGRU_RELU(days, width, height, depth):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (days, depth, height, width)
    chanDim = [1, 2]
    
    # define the model input    
    img = tf.keras.Input(shape=inputShape, name='Input_Layer')
    mask = tf.keras.Input(shape=(1,1,32,32))
    extra = tf.keras.Input(shape=(13))
    
    
    x = tf.keras.layers.experimental.preprocessing.Normalization(axis=2)(img)
    
    conv_2d_layer = tf.keras.layers.Conv2D(32, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)

    conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)

    x = x * mask    
    x = tf.math.reduce_sum(x, axis = [3,4])
    mask_sum = tf.math.reduce_sum(mask, axis = [3,4])
    x = (x / mask_sum)
    
    x = Flatten()(x)
    
    y = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(extra)
    y = BatchNormalization()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)

    x = Dense(128, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(10, kernel_initializer = 'HeUniform')(x)
    
    
    # construct the CNN
    model = DropMaskModel(inputs=[img, mask, extra], outputs=x)
    # return the CNN

    return model

cnn = create_cnn_temp_woGRU_RELU(8, 32, 32, 15)
#cnn.compile(optimizer = 'adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))
print(cnn.summary())

Model: "drop_mask_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 8, 15, 32, 3 0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 8, 15, 32, 32 31          Input_Layer[0][0]                
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 8, 32, 32, 32 4352        normalization[0][0]              
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 32, 32, 32 0           time_distributed[0][0]           
____________________________________________________________________________________

In [22]:
def create_cnn_temp_woGRU(days, width, height, depth):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (days, depth, height, width)
    chanDim = [1, 2]
    
    # define the model input    
    img = tf.keras.Input(shape=inputShape, name='Input_Layer')
    mask = tf.keras.Input(shape=(1,1,32,32))
    extra = tf.keras.Input(shape=(12))
    
    conv_2d_layer = tf.keras.layers.Conv2D(16, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(img)
    x = BatchNormalization(axis=chanDim)(x)
    
    #max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_first')
    #x = tf.keras.layers.TimeDistributed(max_pool_layer)(x)
    
    
    conv_2d_layer = tf.keras.layers.Conv2D(32, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = BatchNormalization(axis=chanDim)(x)
    #max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_first')
    #x = tf.keras.layers.TimeDistributed(max_pool_layer)(x)
    
    
    conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = BatchNormalization(axis=chanDim)(x)

    x = x * mask    
    x = tf.math.reduce_sum(x, axis = [3,4])
    mask_sum = tf.math.reduce_sum(mask, axis = [3,4])
    x = (x / mask_sum)
    
    #gru = tf.keras.layers.GRU(64, return_sequences=False, return_state=False)
    #x = gru(x)
    
    x = Flatten()(x)
    
    y = BatchNormalization()(extra)
    x = tf.keras.layers.concatenate([x, y])
    
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(256, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(10, kernel_initializer = 'HeUniform')(x)
    
    
    # construct the CNN
    model = DropMaskModel(inputs=[img, mask, extra], outputs=x)
    # return the CNN

    return model

cnn = create_cnn_temp_woGRU(8, 32, 32, 15)
#cnn.compile(optimizer = 'adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))
print(cnn.summary())

Model: "drop_mask_model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 8, 15, 32, 3 0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 8, 16, 32, 32 2176        Input_Layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 8, 16, 32, 32 512         time_distributed_6[0][0]         
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 8, 32, 32, 32 4640        batch_normalization_8[0][0]      
__________________________________________________________________________________

In [11]:
def create_cnn_GRU_norm(days, width, height, depth):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (days, depth, height, width)
    chanDim = [1, 2]
    
    # define the model input    
    img = tf.keras.Input(shape=inputShape, name='Input_Layer')
    mask = tf.keras.Input(shape=(1,1,32,32))
    extra = tf.keras.Input(shape=(12))
    
    
    x = tf.keras.layers.experimental.preprocessing.Normalization(axis=2)(img)
    
    conv_2d_layer = tf.keras.layers.Conv2D(32, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    #max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), data_format='channels_first')
    #x = tf.keras.layers.TimeDistributed(max_pool_layer)(x)
    
    
    conv_2d_layer = tf.keras.layers.Conv2D(64, (3, 3), data_format='channels_first', padding='same')
    x = tf.keras.layers.TimeDistributed(conv_2d_layer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)

    x = x * mask    
    x = tf.math.reduce_sum(x, axis = [3,4])
    mask_sum = tf.math.reduce_sum(mask, axis = [3,4])
    x = (x / mask_sum)
    
    gru = tf.keras.layers.GRU(64, return_sequences=False, return_state=False)
    x = gru(x)
    
    
    y = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(extra)
    
    x = tf.keras.layers.concatenate([x, y])
    
    
    x = Dense(128, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(128, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    
    x = Dense(32, kernel_initializer = 'HeUniform')(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(10, kernel_initializer = 'HeUniform')(x)
    
    
    # construct the CNN
    model = DropMaskModel(inputs=[img, mask, extra], outputs=x)
    # return the CNN

    return model

cnn = create_cnn_GRU_norm(8, 32, 32, 15)
#cnn.compile(optimizer = 'adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))
print(cnn.summary())

Model: "drop_mask_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 8, 15, 32, 3 0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 8, 15, 32, 32 31          Input_Layer[0][0]                
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 8, 32, 32, 32 4352        normalization[0][0]              
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 32, 32, 32 0           time_distributed[0][0]           
____________________________________________________________________________________

In [62]:
#tiles_test_dataset = tf.data.Dataset.list_files(f'/home/jupyter/NF-Capstone-Crop-Classification/stacked_files_CNN_test/*', shuffle=False)

#fields_test_mean_var = pd.read_csv('data/mean_var_8days_test.csv')[['field_id', 'elevation', 'field_area_km2', 'sun_rate','neighboor_label_1', 'neighboor_label_2','neighboor_label_3',
                                #'neighboor_label_4','neighboor_label_5', 'neighboor_label_6','neighboor_label_7', 'neighboor_label_8', 'neighboor_label_9']]

#tiles_test = tilu.read_tile_geojson('data/tiles_train.geojson')
#tiles_test_crs32634 = tiles_test.to_crs(32634)

# apply with a threshold of 4000m
#tiles_test['tiles_closest'] = tiles_test_crs32634.apply(tilu.tiles_closest, axis='columns', args=(tiles_train_crs32634, 4000,))

for i, f in enumerate(tiles_test_dataset.as_numpy_iterator()):
    if i == 0:
        (test_ds, field_ids) = tile_dataset(f, train=False, radius=16, extra_data=fields_test_mean_var, tiles_info=tiles_test)
    else:
        (test_ds_append, field_ids_append) = tile_dataset(f, train=False, radius=16, extra_data=fields_test_mean_var, tiles_info=tiles_test)
        test_ds = test_ds.concatenate(test_ds_append)
        field_ids.extend(field_ids_append)

999


In [63]:
AUTOTUNE = tf.data.AUTOTUNE

batch_size = 1024

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    return ds

test_ds = configure_for_performance(test_ds)

In [65]:
inputShape = (8, 15, 32, 32)
img = tf.keras.Input(shape=inputShape, name='Input_Layer')
mask = tf.keras.Input(shape=(1,1,32,32))
extra = tf.keras.Input(shape=(13))

x = cnn([img, mask, extra], training=False)
x = tf.keras.layers.Softmax()(x)

prob_model = DropMaskModel(inputs=[img, mask, extra], outputs=x)


prob_model.compile(optimizer = 'adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

In [34]:
cnn.evaluate(val_ds, verbose = 1)

22/22 [==============================] - 38s 1s/step - loss: 0.8350 - accuracy: 0.7034


[0.8349533081054688, 0.7033804655075073]

In [66]:
ds_test_pred_proba = prob_model.predict(test_ds, verbose=1)

35/35 [==============================] - 81s 2s/step


In [67]:
ds_test_pred_proba.shape

(35295, 10)

In [68]:
##### Build Submission
df_field_ids = pd.DataFrame(field_ids)

In [76]:
df_sol_proba = pd.DataFrame(((ds_test_pred_proba + 0.01)/np.sum((ds_test_pred_proba + 0.01), axis=1, keepdims=True)).round(2))
#df_sol_proba = pd.DataFrame(ds_test_pred_proba.round(2))

In [77]:
df_sol = pd.concat([df_field_ids,df_sol_proba], axis=1)

In [78]:
df_sol.columns = ['field_id',0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
df_sol

,field_id,0,1,2,3,4,5,6,7,8,9
0,2199,0.00,0.00,0.00,0.01,0.98,0.00,0.00,0.00,0.00,0.00
1,2277,0.00,0.04,0.08,0.08,0.74,0.04,0.01,0.01,0.00,0.00
2,2571,0.00,0.06,0.17,0.09,0.59,0.05,0.01,0.01,0.00,0.00
3,3112,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
4,7476,0.00,0.04,0.10,0.09,0.72,0.03,0.01,0.01,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
35290,115450,0.01,0.12,0.36,0.07,0.06,0.07,0.16,0.09,0.05,0.01
35291,116014,0.00,0.25,0.38,0.07,0.06,0.07,0.09,0.04,0.02,0.02
35292,120968,0.00,0.06,0.09,0.03,0.02,0.03,0.28,0.44,0.05,0.01
35293,122334,0.00,0.14,0.55,0.08,0.10,0.07,0.03,0.02,0.01,0.01


In [79]:
df_sol = df_sol[["field_id", 8, 3, 1,2,9,6,5,7,4]]
column_names = {"field_id": "Field ID",8:"Crop_Canola",3:"Crop_Fallow",1:"Crop_Lucerne/Medics",2:"Crop_Planted pastures (perennial)",9:"Crop_Rooibos",6:"Crop_Small grain grazing",5:"Crop_Weeds",7:"Crop_Wheat",4:"Crop_Wine grapes"}

df_sol_clean = df_sol.rename(columns = column_names)
df_sol_clean

,Field ID,Crop_Canola,Crop_Fallow,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Rooibos,Crop_Small grain grazing,Crop_Weeds,Crop_Wheat,Crop_Wine grapes
0,2199,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.98
1,2277,0.00,0.08,0.04,0.08,0.00,0.01,0.04,0.01,0.74
2,2571,0.00,0.09,0.06,0.17,0.00,0.01,0.05,0.01,0.59
3,3112,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
4,7476,0.00,0.09,0.04,0.10,0.00,0.01,0.03,0.01,0.72
...,...,...,...,...,...,...,...,...,...,...
35290,115450,0.05,0.07,0.12,0.36,0.01,0.16,0.07,0.09,0.06
35291,116014,0.02,0.07,0.25,0.38,0.02,0.09,0.07,0.04,0.06
35292,120968,0.05,0.03,0.06,0.09,0.01,0.28,0.03,0.44,0.02
35293,122334,0.01,0.08,0.14,0.55,0.01,0.03,0.07,0.02,0.10


In [81]:
df_sol_clean.to_csv("Submission_LSTM-10Epochs.csv", index=False, float_format='%.2f')

In [325]:
df_sol_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35295 entries, 0 to 35294
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Field ID                           35295 non-null  int32  
 1   Crop_Canola                        35295 non-null  float32
 2   Crop_Fallow                        35295 non-null  float32
 3   Crop_Lucerne/Medics                35295 non-null  float32
 4   Crop_Planted pastures (perennial)  35295 non-null  float32
 5   Crop_Rooibos                       35295 non-null  float32
 6   Crop_Small grain grazing           35295 non-null  float32
 7   Crop_Weeds                         35295 non-null  float32
 8   Crop_Wheat                         35295 non-null  float32
 9   Crop_Wine grapes                   35295 non-null  float32
dtypes: float32(9), int32(1)
memory usage: 1.3 MB
